In [1]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

unlp_2025_shared_task_classification_techniques_path = kagglehub.competition_download('unlp-2025-shared-task-classification-techniques')
oleksandrkharytonov_translated_train_unlp_2025_path = kagglehub.dataset_download('oleksandrkharytonov/translated-train-unlp-2025')

print('Data source import complete.')
print(unlp_2025_shared_task_classification_techniques_path)
print(oleksandrkharytonov_translated_train_unlp_2025_path)

100%|██████████| 4.48M/4.48M [00:00<00:00, 125MB/s]

Extracting files...


100%|██████████| 7.80M/7.80M [00:00<00:00, 155MB/s]

Extracting files...


Data source import complete.
/root/.cache/kagglehub/competitions/unlp-2025-shared-task-classification-techniques
/root/.cache/kagglehub/datasets/oleksandrkharytonov/translated-train-unlp-2025/versions/4


In [3]:
!pip install -q -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.1 MB/s eta 0:00:00


In [5]:
import huggingface_hub
import wandb

huggingface_hub.login(token=hugginface_key)
wandb.login(key=wandb_key)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shah1st-work-ua (shah1st-work-ua-igor-sikorsky-kyiv-polytechnic-institute) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [6]:
wandb.init(
    project="unlp-clf-task",
    entity="shah1st-work-ua-igor-sikorsky-kyiv-polytechnic-institute",
    name='aya-8b-baseline',
    #init_timeout=240
)

In [7]:
import pandas as pd


data_path = unlp_2025_shared_task_classification_techniques_path
cv_path: str = oleksandrkharytonov_translated_train_unlp_2025_path + "/cv_split.csv"


df = pd.read_parquet(data_path + "/train.parquet")
cv = pd.read_csv(cv_path)
df = df.merge(cv, on='id', how='left')

test = pd.read_csv(data_path + "/test.csv")

In [8]:
import numpy as np

df['is_valid'] = df.fold == 4

In [9]:
from tqdm.autonotebook import tqdm

tqdm.pandas()
from transformers import pipeline, AutoTokenizer

PRETRAINED_MODEL = 'CohereForAI/aya-expanse-8b'
MAX_LENGTH = 2048

tokenizer = AutoTokenizer.from_pretrained(
    PRETRAINED_MODEL
)

tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)
tokenizer.padding_side = 'right'
tokenizer.add_eos_token = True

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/8.64k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/12.8M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

In [10]:
import json

def prompt_generator(text):
    messages = [
        {
            "role": "user",
            "content": f"Ти експерт в аналізу постів в соцмережах. Тобі необхідно визначити наскільки пости містять в собі техніки маніпуляції.\nТекст статті: {text}"
        }
    ]

    conversation = tokenizer.apply_chat_template(messages, tokenize=True)
    # return conversation
    return tokenizer.decode(conversation)

In [11]:
prompt_generator("mama")

'<BOS_TOKEN><|START_OF_TURN_TOKEN|><|USER_TOKEN|>Ти експерт в аналізу постів в соцмережах. Тобі необхідно визначити наскільки пости містять в собі техніки маніпуляції.\nТекст статті: mama<|END_OF_TURN_TOKEN|>'

In [12]:
df.loc[:, 'prompt'] = df.content.apply(prompt_generator)
test.loc[:, 'prompt'] = test.content.apply(prompt_generator)

In [13]:
df['full_text'] = df.prompt.progress_apply(
    lambda x: tokenizer.decode(tokenizer(x, add_special_tokens=False)['input_ids'][:MAX_LENGTH])
)
test['full_text'] = test.prompt.progress_apply(
    lambda x: tokenizer.decode(tokenizer(x, add_special_tokens=False)['input_ids'][:MAX_LENGTH])
)

def tokenize(sample):
    tokenized = tokenizer(sample['full_text'])
    return tokenized

  0%|          | 0/3822 [00:00<?, ?it/s]

  0%|          | 0/5735 [00:00<?, ?it/s]

In [14]:
ssubmission = pd.read_csv(data_path + '/sample_submission.csv')
targets = ssubmission.set_index('id').columns

from collections.abc import Iterable

for col in targets:
    df[col] = 0

import numpy as np
for ind, row in df.iterrows():
    if isinstance(row['techniques'], Iterable):
        for t in row['techniques']:
            df.loc[ind, t] = 1

In [15]:
df['labels'] = list(df[targets].values)

In [16]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.


In [17]:
from datasets import Dataset

ds_train = Dataset.from_pandas(df[df.is_valid == 0][['full_text', 'labels']].copy())
ds_eval = Dataset.from_pandas(df[df.is_valid == 1][['full_text', 'labels']].copy())
ds_test = Dataset.from_pandas(test[['full_text']].copy())

ds_train = ds_train.map(tokenize)
remove_columns = [c for c in ds_train.features.keys() if c not in ['input_ids', 'attention_mask', 'labels']]
ds_train = ds_train.remove_columns(remove_columns)

ds_eval = ds_eval.map(tokenize)
remove_columns = [c for c in ds_eval.features.keys() if c not in ['input_ids', 'attention_mask', 'labels']]
ds_eval = ds_eval.remove_columns(remove_columns)

ds_test = ds_test.map(tokenize)
remove_columns = [c for c in ds_test.features.keys() if c not in ['input_ids', 'attention_mask', 'labels']]
ds_test = ds_test.remove_columns(remove_columns)

Map:   0%|          | 0/3058 [00:00<?, ? examples/s]

Map:   0%|          | 0/764 [00:00<?, ? examples/s]

Map:   0%|          | 0/5735 [00:00<?, ? examples/s]

In [18]:
import torch
import torch.nn as nn
from typing import Optional, Tuple, Union
from transformers import CoherePreTrainedModel, CohereModel
from transformers.modeling_outputs import SequenceClassifierOutputWithPast
from transformers.utils import add_start_docstrings_to_model_forward

class CohereForSequenceClassification(CoherePreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.model = CohereModel(config)
        self.score = nn.Linear(config.hidden_size, self.num_labels, bias=False)

        # Initialize weights and apply final processing
        self.post_init()

    def get_input_embeddings(self):
        return self.model.embed_tokens

    def set_input_embeddings(self, value):
        self.model.embed_tokens = value


    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[Tuple] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, SequenceClassifierOutputWithPast]:
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        transformer_outputs = self.model(
            input_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = transformer_outputs[0]
        logits = self.score(hidden_states)

        if input_ids is not None:
            batch_size = input_ids.shape[0]
        else:
            batch_size = inputs_embeds.shape[0]

        if self.config.pad_token_id is None and batch_size != 1:
            raise ValueError("Cannot handle batch sizes > 1 if no padding token is defined.")
        if self.config.pad_token_id is None:
            last_non_pad_token = -1
        elif input_ids is not None:
            non_pad_mask = (input_ids != self.config.pad_token_id).to(logits.device, torch.int32)
            token_indices = torch.arange(input_ids.shape[-1], device=logits.device, dtype=torch.int32)
            last_non_pad_token = (token_indices * non_pad_mask).argmax(-1)
        else:
            last_non_pad_token = -1

        pooled_logits = logits[torch.arange(batch_size, device=logits.device), last_non_pad_token]

        loss = None
        if labels is not None:
            loss = self.loss_function(logits=logits, labels=labels, pooled_logits=pooled_logits, config=self.config)

        if not return_dict:
            output = (pooled_logits,) + transformer_outputs[1:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutputWithPast(
            loss=loss,
            logits=pooled_logits,
            past_key_values=transformer_outputs.past_key_values,
            # hidden_states=transformer_outputs.hidden_states,
            attentions=transformer_outputs.attentions,
        )

In [20]:
import torch
from transformers import BitsAndBytesConfig
from peft import get_peft_config, prepare_model_for_kbit_training, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType


model = CohereForSequenceClassification.from_pretrained(
    PRETRAINED_MODEL,
    num_labels=10,
    torch_dtype=torch.bfloat16
)

lora_config = LoraConfig(
    r=32,  # the dimension of the low-rank matrices
    lora_alpha=16, # scaling factor for LoRA activations vs pre-trained weight activations
    lora_dropout=0.05,
    bias='none',
    inference_mode=False,
    task_type=TaskType.SEQ_CLS,
    target_modules=['o_proj', 'v_proj', "q_proj", "k_proj", "gate_proj"]
)

model = get_peft_model(model, lora_config)
# Trainable Parameters
model.print_trainable_parameters()



config.json:   0%|          | 0.00/634 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of CohereForSequenceClassification were not initialized from the model checkpoint at CohereForAI/aya-expanse-8b and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 46,178,304 || all params: 8,074,252,288 || trainable%: 0.5719


In [21]:
import os
import random


def set_seeds(seed):
    """Set seeds for reproducibility """
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)


set_seeds(seed=42)

In [22]:
from scipy.optimize import minimize_scalar

def find_thresholds_for_distribution(preds, desired_distribution):
    """
    Find thresholds for each class to achieve the desired class distribution.

    Args:
        preds (ndarray): Array of shape (num_samples, num_classes) with probabilities (after sigmoid).
        desired_distribution (list): Desired proportion of positive samples for each class.

    Returns:
        thresholds (list): List of thresholds for each class.
    """
    num_classes = preds.shape[1]
    thresholds = []

    for class_idx in range(num_classes):
        probs = preds[:, class_idx]
        desired_ratio = desired_distribution[class_idx]

        # Function to minimize the difference between actual and desired positive ratios
        def objective(threshold):
            predicted_ratio = (probs >= threshold).mean()
            return abs(predicted_ratio - desired_ratio)

        # Find the threshold using optimization
        result = minimize_scalar(objective, bounds=(0, 1), method="bounded")
        thresholds.append(result.x)

    return thresholds
df[targets].mean().values

array([0.03610675, 0.07849294, 0.1007326 , 0.04107797, 0.04133961,
       0.51622187, 0.12637363, 0.12087912, 0.13396128, 0.12114076])

In [23]:
import os
from transformers import (AutoTokenizer, TrainingArguments, Trainer,
                          AutoModelForSequenceClassification, DataCollatorWithPadding)
from sklearn.metrics import f1_score
import numpy as np


TARGET_DISTRIBUTION = df[targets].mean().values


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    proba = torch.nn.functional.sigmoid(torch.tensor(logits)).numpy()
    optimal_thresholds = find_thresholds_for_distribution(
        proba, desired_distribution=TARGET_DISTRIBUTION
    )
    binarized_preds = (proba >= np.array(optimal_thresholds)).astype(int)

    return {"f1": f1_score(labels, binarized_preds, average="macro")}


train_args = TrainingArguments(
    output_dir='model_checkpoints_aya8b_qlora_tholds',
    logging_dir='./model_logs_aya8b_qlora_tholds',
    learning_rate=2e-5,
    weight_decay=0.01,
    lr_scheduler_type='cosine',
    warmup_ratio=0.0,
    num_train_epochs=4,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    bf16=True,
    report_to="wandb",
    optim='adamw_torch',
    eval_strategy='steps',
    save_strategy="steps",
    eval_steps=200,
    logging_steps=20,
    save_steps=200,
    save_total_limit=10,
    metric_for_best_model='f1',
    greater_is_better=True,
    load_best_model_at_end=True,
)

In [24]:
from torch.nn import BCEWithLogitsLoss

class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = torch.tensor((1/df[targets].mean()).tolist()).cuda()

    def compute_loss(self, model, inputs, return_outputs=False, *args, **kwargs):
        outputs = model(inputs['input_ids'], inputs['attention_mask'])
        logits = outputs.logits

        # Initialize BCEWithLogitsLoss with class weights
        #loss_fn = BCEWithLogitsLoss(weight=self.class_weights)
        loss_fn = BCEWithLogitsLoss()
        #print(logits[:2])
        #print(inputs['labels'][:2])
        loss = loss_fn(logits, inputs['labels'].float())

        return (loss, outputs) if return_outputs else loss

In [25]:
trainer = CustomTrainer(
    model=model,
    args=train_args,
    train_dataset=ds_train,
    eval_dataset=ds_eval,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics
)
trainer.train()

<ipython-input-24-253d12176855>:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss,F1
200,0.285200,0.321617,0.238992
400,0.255900,0.288053,0.250245
600,0.281200,0.272385,0.336134
800,0.243400,0.262152,0.367118
1000,0.265700,0.260019,0.375096
1200,0.244900,0.255094,0.382964
1400,0.220700,0.255272,0.397903


TrainOutput(global_step=1528, training_loss=0.28311609394887355, metrics={'train_runtime': 1878.9774, 'train_samples_per_second': 6.51, 'train_steps_per_second': 0.813, 'total_flos': 1.5662390733707674e+17, 'train_loss': 0.28311609394887355, 'epoch': 3.991497710922171})

In [26]:
preds_val = trainer.predict(ds_eval)

In [27]:
import numpy as np
from sklearn.metrics import f1_score

def optimize_thresholds(preds, targets, num_thresholds=100):
    """
    Find optimal thresholds for each class to maximize average F1 score.

    Args:
        preds (ndarray): Array of shape (num_samples, num_classes) with logits.
        targets (ndarray): Array of shape (num_samples, num_classes) with binary ground-truth labels.
        num_thresholds (int): Number of thresholds to evaluate (default: 100).

    Returns:
        optimal_thresholds (list): List of optimal thresholds for each class.
        best_avg_f1 (float): Best average F1 score achieved.
    """
    num_classes = preds.shape[1]
    thresholds = np.linspace(0, 1, num_thresholds)
    optimal_thresholds = []
    best_avg_f1 = 0

    for class_idx in tqdm(range(num_classes)):
        best_f1 = 0
        best_threshold = 0
        for threshold in thresholds:
            # Binarize predictions for this class
            binarized_preds = (preds[:, class_idx] >= threshold).astype(int)
            # Calculate F1 score for this class
            f1 = f1_score(targets[:, class_idx], binarized_preds, zero_division=0)
            if f1 > best_f1:
                best_f1 = f1
                best_threshold = threshold
        optimal_thresholds.append(best_threshold)

    # Calculate average F1 score across all classes using optimal thresholds
    binarized_preds = (preds >= np.array(optimal_thresholds)).astype(int)
    avg_f1 = f1_score(targets, binarized_preds, average='macro', zero_division=0)

    return optimal_thresholds, avg_f1

In [28]:
optimal_thresholds, best_avg_f1 = optimize_thresholds(
    torch.nn.functional.sigmoid(
        torch.tensor(preds_val.predictions)).numpy(), preds_val.label_ids)

  0%|          | 0/10 [00:00<?, ?it/s]

In [29]:
best_avg_f1

0.43488966782138805

In [30]:
optimal_thresholds_notnull = np.array(optimal_thresholds) + 1e-6

In [31]:
binarized_preds = (torch.nn.functional.sigmoid(
        torch.tensor(preds_val.predictions)).numpy() >= np.array(optimal_thresholds_notnull)).astype(int)

In [32]:
a = f1_score(preds_val.label_ids, binarized_preds, average='macro', zero_division=0)

In [33]:
a

0.43488966782138805

In [34]:
preds_test = trainer.predict(ds_test)

In [35]:
binarized_preds = (torch.nn.functional.sigmoid(
        torch.tensor(preds_test.predictions)).numpy() >= np.array(optimal_thresholds_notnull)).astype(int)

In [36]:
for ind, col in enumerate(ssubmission.columns[1:]):
    ssubmission[col] = binarized_preds[:, ind]

In [37]:
ssubmission.to_csv(f'aya8b-cv{round(a,2)}.csv', index=False)

In [38]:
import numpy as np
from scipy.optimize import minimize_scalar

def find_thresholds_for_distribution(preds, desired_distribution):
    """
    Find thresholds for each class to achieve the desired class distribution.

    Args:
        preds (ndarray): Array of shape (num_samples, num_classes) with probabilities (after sigmoid).
        desired_distribution (list): Desired proportion of positive samples for each class.

    Returns:
        thresholds (list): List of thresholds for each class.
    """
    num_classes = preds.shape[1]
    thresholds = []

    for class_idx in range(num_classes):
        probs = preds[:, class_idx]
        desired_ratio = desired_distribution[class_idx]

        # Function to minimize the difference between actual and desired positive ratios
        def objective(threshold):
            predicted_ratio = (probs >= threshold).mean()
            return abs(predicted_ratio - desired_ratio)

        # Find the threshold using optimization
        result = minimize_scalar(objective, bounds=(0, 1), method="bounded")
        thresholds.append(result.x)

    return thresholds

In [39]:
df[targets].mean().values

array([0.03610675, 0.07849294, 0.1007326 , 0.04107797, 0.04133961,
       0.51622187, 0.12637363, 0.12087912, 0.13396128, 0.12114076])

In [40]:
optimal_thresholds = find_thresholds_for_distribution(
    torch.nn.functional.sigmoid(
        torch.tensor(preds_val.predictions)).numpy(), desired_distribution=df[targets].mean().values)

In [41]:
binarized_preds = (torch.nn.functional.sigmoid(
        torch.tensor(preds_val.predictions)).numpy() >= np.array(optimal_thresholds)).astype(int)

In [42]:
b = f1_score(preds_val.label_ids, binarized_preds, average='macro', zero_division=0)

In [43]:
b

0.3979033901115048

In [44]:
binarized_preds = (torch.nn.functional.sigmoid(
        torch.tensor(preds_test.predictions)).numpy() >= np.array(optimal_thresholds)).astype(int)

In [45]:
binarized_preds.mean(axis=0)

array([0.03801221, 0.06800349, 0.10305144, 0.03138622, 0.04690497,
       0.52728858, 0.1299041 , 0.12414996, 0.13513514, 0.12763731])

In [46]:
for ind, col in enumerate(ssubmission.columns[1:]):
    ssubmission[col] = binarized_preds[:, ind]

In [47]:
ssubmission.to_csv(f'aya8b-cv{round(b,2)}.csv', index=False)